In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import ast
from tqdm import tqdm
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics 


import matplotlib.pyplot as plt

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc'),
]

# make the data set

In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/train_full_feature.csv")
test_df = pd.read_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/test_full_feature.csv")
data=train_df[["common_interest_rank","common_friends","distance_inv","sink_degree"]].astype(float)
classlabel=train_df['label']
X_train, X_test, y_train, y_test = train_test_split(data,classlabel, train_size=0.70, test_size=0.30, random_state = 6010)

In [ ]:
test_df = pd.read_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/test_full_feature.csv")
test=np.array(test_df[["common_interest_rank","common_friends","distance_inv","sink_degree"]].astype(float))

# Logistic regression

## visualise the data

In [ ]:
X = np.array(df_col[["common_interest","common_friends","distance","source_degree","sink_degree"]])
y = np.array(list(df_col["label"]))

In [ ]:
X_b = np.column_stack((np.ones_like(y), X))

In [ ]:
import seaborn as sns
from scipy.special import expit
sns.set_style('darkgrid')
plt.rcParams['figure.dpi'] = 108

## fit logistic regression 

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(penalty='l2')
clf.fit(X_train, y_train)
w_sklearn = np.r_[clf.intercept_, clf.coef_.squeeze()]
print("Weights according to scikit-learn: {}".format(w_sklearn))

In [ ]:
X_train

In [ ]:
y_pred=clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
?LogisticRegression

# Neural Network

## NN 1

In [ ]:

# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = METRICS)
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=120, batch_size=10, validation_data=(X_test, y_test))
# evaluate the keras model
#_, accuracy = model.evaluate(X_train, y_train)
#print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
predictions_test = model.predict(test)
out = pd.read_csv('/content/drive/My Drive/COMP90051_project1/Features/test_common_interest_2.csv')
out[["Predicted"]] = predictions_test
out.to_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/NN_4f_1.csv", index = False)

## NN

In [ ]:

# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# define the keras model
model = Sequential()
model.add(Dense(10, input_dim=4, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=8, validation_data=(X_test, y_test))

In [ ]:
predictions_test = model.predict(test)
out = pd.read_csv('/content/drive/My Drive/COMP90051_project1/Features/test_common_interest_2.csv')
out[["Predicted"]] = predictions_test
out.to_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/NN_4f_2.csv", index = False)

## CNN

In [ ]:
x_train_reshape = np.expand_dims(X_train, -1)
x_train_reshape = np.expand_dims(x_train_reshape, -1)
#y_train_reshape = np.expand_dims(y_train, -1)
#y_train_reshape = np.expand_dims(y_train_reshape, -1)
x_test_reshape = np.expand_dims(X_test, -1)
x_test_reshape = np.expand_dims(x_test_reshape, -1)
#y_test_reshape = np.expand_dims(y_test, -1)
#y_test_reshape = np.expand_dims(y_test_reshape, -1)


In [ ]:
test_kaggle_reshape = np.expand_dims(np.array(test), -1)
test_kaggle_reshape = np.expand_dims(test_kaggle_reshape, -1)

In [ ]:
test_kaggle_reshape.shape

In [ ]:
cnn = keras.Sequential(
    [
        layers.Input((4, 1, 1)),
        layers.Conv2D(8, (2, 1), activation='relu'), # fill in
        layers.MaxPooling2D((1, 1)), # fill in
        layers.Conv2D(8, (2, 1), activation='relu'), #fill in
        layers.MaxPooling2D((1, 1)), # fill in
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ], 
)

In [ ]:
cnn.compile(optimizer='adam',
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=METRICS)

In [ ]:
history_cnn = cnn.fit(x_train_reshape, y_train, epochs=150, batch_size=10, 
                      validation_data=(x_test_reshape, y_test))

In [ ]:
cnn.fit

In [ ]:
predictions_test = cnn.predict(test_kaggle_reshape)
out = pd.read_csv('/content/drive/My Drive/COMP90051_project1/Features/test_common_interest_2.csv')
out["Predicted"] = predictions_test
out.to_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/CNN_4f_1.csv", index = False)

# Random Forest

##RF1

In [ ]:
# set up
clf = RandomForestClassifier(n_estimators=100,max_depth=4)
clf.fit(X_train,y_train)
y_train_pred=clf.predict(X_train)
y_test_pred=clf.predict(X_test)
print("Train Accuracy:",accuracy_score(y_train, y_train_pred))
print("Train AUC:",roc_auc_score(y_train, y_train_pred))
print("Test Accuracy:",accuracy_score(y_test, y_test_pred))
print("Test AUC:",roc_auc_score(y_test, y_test_pred))
feature_imp = pd.Series(clf.feature_importances_).sort_values(ascending=False)
feature_imp

In [ ]:
predictions_test = clf.predict_proba(test)
out = pd.read_csv('/content/drive/My Drive/COMP90051_project1/Features/test_common_interest_2.csv')
out["Predicted"] = predictions_test
out.to_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/RF_4f_1.csv", index = False)

In [ ]:
len(out)

##RF2

In [ ]:
# set up
clf = RandomForestClassifier(n_estimators=100,max_depth=6, criterion='entropy')
clf.fit(X_train,y_train)
y_train_pred=clf.predict(X_train)
y_test_pred=clf.predict(X_test)
print("Train Accuracy:",accuracy_score(y_train, y_train_pred))
print("Train AUC:",roc_auc_score(y_train, y_train_pred))
print("Test Accuracy:",accuracy_score(y_test, y_test_pred))
print("Test AUC:",roc_auc_score(y_test, y_test_pred))
feature_imp = pd.Series(clf.feature_importances_).sort_values(ascending=False)
feature_imp

In [ ]:
predictions_test = clf.predict_proba(test)
out = pd.read_csv('/content/drive/My Drive/COMP90051_project1/Features/test_common_interest_2.csv')
out["Predicted"] = predictions_test
out.to_csv("/content/drive/My Drive/COMP90051_project1/new_train_model/RF_4f_2.csv", index = False)